## 3. Recurrent Neural Network

- feedforward 
- backpropation
- sequence 계열 데이터에 사용되는 모델

### data

### reference : cs231n RNN 참고

In [2]:
# import module
import numpy as np

In [1]:
# data
# reference : https://github.com/aisolab/CS20
sentences = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
# 형태소
pos = [['pronoun', 'verb', 'adjective'],
     ['noun', 'verb', 'adverb', 'adjective'],
     ['noun', 'verb', 'determiner', 'noun', 'preposition', 'adjective', 'noun'],
     ['noun', 'verb', 'adverb', 'adjective', 'verb']]


In [5]:
# word dict
word_list = []
for elm in sentences:
    word_list += elm
word_list = list(set(word_list)) # unique한 word만 list
word_list.sort()
word_list = ['<pad>'] + word_list # '<pad>' 추가
word_to_ix = {word : idx for idx, word in enumerate(word_list)}  # word
ix_to_word={idx: word for idx, word in enumerate(word_list)}

In [6]:
# 확인하기
print(word_to_ix)
print(ix_to_word)

{'<pad>': 0, 'I': 1, 'a': 2, 'changing': 3, 'deep': 4, 'difficult': 5, 'fast': 6, 'feel': 7, 'for': 8, 'framework': 9, 'hungry': 10, 'is': 11, 'learning': 12, 'tensorflow': 13, 'very': 14}
{0: '<pad>', 1: 'I', 2: 'a', 3: 'changing', 4: 'deep', 5: 'difficult', 6: 'fast', 7: 'feel', 8: 'for', 9: 'framework', 10: 'hungry', 11: 'is', 12: 'learning', 13: 'tensorflow', 14: 'very'}


In [10]:
# size
data_size, vocab_size=len([ word for i in sentences for word in i]), len(word_to_ix)


19


In [11]:
# hyperparameter
hidden_size=100 # size of hidden layer of neurons
seq_length=25 # number of steps to unroll the RNN
learning_rate=1e-1

# model parameter
Wxh = np.random.randn(hidden_size,vocab_size)*0.01 # input to hidden
Whh= np.random.randn(hidden_size,hidden_size)*0.01 # hidden to hidden
Why= np.random.randn(vocab_size,hidden_size)*0.01 # hiddent to output
bh =np.zeros((hidden_size,1))
by = np.zeros((vocab_size,1))

In [ ]:
# sampling
def sample(h,seed_ix,n):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    return sample_ix
    """
    
    x=np.zeros((vocab_size,1)) # 가능한 정답지 vocab
    x[seed_ix]=1 # seed_ix에 해당하는 index만 1 나머지는 모두 0
    ixes=[]
    
    """ 
    xrange 타입은 수정이 불가한 순차적 접근 가능한 데이터 타입이다. 
    xrange 타입의 장점이라고 하면 지정한 데이터 크기에 상관없이 memory 할당량이 일정하다는 것이다.
    """
    for t in xrange(n):
        h=np.tanh(np.dot(Wxh,x)+np.dot(Whh,h)+bh) # next hidden state
        y=np.dot(Why,h)+by # y값 예측
        p=np.exp(y)/np.sum(np.exp(y)) # print(p) 해보기
        # np.random.choice : p를 지정하면 각 p에 속하는 확률로 sampling
        ix=np.random.choice(range(vocab_size),p=p.ravel()) # ravel : 다차원 배열을 1차원 배열로 평평하게 변환
        x=np.zeros((vocab_size,1))
        x[ix]=1
        ixes.append(ix)
        
    return ixes
        

In [ ]:
# loss function : forward pass
def lossFun(inputs, targets, hprev):
    """
    input : list of integer
    target : list of interger
    hprev :  Hx1 array of initial state
    return the loss, gradient on model parameters, and last hiddent state
    """
    xs, hs, ys, ps={},{},{},{}
    hs[-1]=np.copy(hprev)
    loss=0
    
    # forward pass
    for t in xrange(len(inputs)):
        xs[t]